In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Create synthetic observed data
num_samples = 100000

# Environmental variables (affect occupancy)
env_variables = np.random.rand(num_samples, 3)  # env_var1, env_var2, env_var3
env_df = pd.DataFrame(env_variables, columns=['env_var1', 'env_var2', 'env_var3'])

# Detection variables (affect detection probability)
detect_variables = np.random.rand(num_samples, 2)  # detect_var1, detect_var2
detect_df = pd.DataFrame(detect_variables, columns=['detect_var1', 'detect_var2'])

# Fixed occupancy probability based on environmental variables
occupancy_prob = np.where(env_df['env_var1'] > 0.5, 0.8, 0.3)
occupancy_prob += np.where(env_df['env_var2'] > 0.6, 0.1, 0)
occupancy_prob = np.clip(occupancy_prob, 0, 1)

# Fixed detection probability based on detection variables
detection_prob = np.where(detect_df['detect_var1'] > 0.5, 0.7, 0.4)
detection_prob += np.where(detect_df['detect_var2'] > 0.3, 0.2, 0)
detection_prob = np.clip(detection_prob, 0, 1)

# Combined probability of detection: occupancy_prob * detection_prob
# (optional: can still be occupancy_prob + detection_prob if needed)
combined_prob = occupancy_prob * detection_prob

# Observed detection outcomes (we only observe this)
detection = np.random.binomial(1, combined_prob)

# Combine all data into a single DataFrame
data = pd.concat([env_df, detect_df], axis=1)
data['detection'] = detection
data['true_occupancy_prob'] = occupancy_prob
data['true_detection_prob'] = detection_prob

# Calculate ROC AUC score for evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

auc_score = roc_auc_score(
    np.random.binomial(1, occupancy_prob), detection
)
print("ROC AUC Score:", auc_score)
print("F1 Score:", f1_score(np.random.binomial(1, occupancy_prob), detection))


ROC AUC Score: 0.5914887264547102
F1 Score: 0.5708057938405288


In [4]:
from scipy.special import expit
# Create synthetic observed data
num_samples = 100000

# Environmental variables (affect occupancy)
env_variables = np.random.rand(num_samples, 3)  # env_var1, env_var2, env_var3
env_df = pd.DataFrame(env_variables, columns=['env_var1', 'env_var2', 'env_var3'])

# Detection variables (affect detection probability)
detect_variables = np.random.normal(loc=0.7, scale=0.2, size=(num_samples, 2))  # detect_var1, detect_var2
detect_df = pd.DataFrame(detect_variables, columns=['detect_var1', 'detect_var2'])

# Occupancy probability based on environmental variables
occupancy_prob = (
    0.6 * env_df['env_var1'] +
    0.4 * env_df['env_var2']
)

occupancy_prob = np.array(occupancy_prob).clip(0, 1)
occupancy_prob = expit(20 * (occupancy_prob - 0.5))

# Detection probability based on detection variables
detection_prob = (
    0.7 * detect_df['detect_var1'] +
    0.3 * detect_df['detect_var2']
)
detection_prob = detection_prob.clip(0, 1)

# Observed detection outcomes (we only observe this)
detection = np.random.binomial(1, np.random.binomial(1, occupancy_prob) * detection_prob)

# Combine all data into a single DataFrame
data = pd.concat([env_df, detect_df], axis=1)
data['detection'] = detection
data['true_occupancy_prob'] = occupancy_prob
data['true_detection_prob'] = detection_prob



In [10]:
from sklearn.model_selection import train_test_split
X = data.drop('detection', axis=1)
y = data['detection']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_detection_var_df = X_train[['detect_var1', 'detect_var2']]
X_train_occupancy_var_df = X_train[['env_var1', 'env_var2', 'env_var3']]
X_test_detection_var_df = X_test[['detect_var1', 'detect_var2']]
X_test_occupancy_var_df = X_test[['env_var1', 'env_var2', 'env_var3']]


In [11]:
X_train_combined = pd.concat([X_train_detection_var_df.reset_index(drop=True), X_train_occupancy_var_df.reset_index(drop=True)], axis=1)
X_test_combined = pd.concat([X_test_detection_var_df.reset_index(drop=True), X_test_occupancy_var_df.reset_index(drop=True)], axis=1)


In [83]:
from occupancy_model_nn import occupancy_ml_trainer
model = occupancy_ml_trainer(validation=True, no_mini_batch=False, batch_size=128, tolerance_epoch=10)
model.fit(X_train_combined, y_train)


  epoch    train_f1    train_loss    train_roc_auc    valid_acc    valid_f1    valid_loss    valid_precision    valid_recall    valid_roc_auc     dur
-------  ----------  ------------  ---------------  -----------  ----------  ------------  -----------------  --------------  ---------------  ------
      1      0.7278        0.5009           0.8283       0.7966      0.8127        0.4411             0.7530          0.8826           0.8621  0.6091
      2      0.8161        0.4340           0.8715       0.7966      0.8151        0.4314             0.7474          0.8962           0.8734  0.6071
      3      0.8176        0.4262           0.8772       0.7976      0.8148        0.4263             0.7510          0.8904           0.8755  0.6044
      4      0.8181        0.4228           0.8784       0.7962      0.8158        0.4258             0.7443          0.9023           0.8761  0.6365
      5      0.8173        0.4220           0.8788       0.7991      0.8142        0.4248           

In [89]:
probs = model.predict_proba(X_train_combined)[:,1]
print(roc_auc_score(y_train.values.astype('float32'), probs))
print(f1_score(y_train.values.astype('float32'), np.where(probs>0.5, 1, 0)))



0.8789541561874539
0.7351394924996495
